In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%pylab

from Xana import Xana
import numpy as np
import os
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
import seaborn as sns
# import h5py
# import pandas as pd

# Functions

In [ ]:
def mask_streaks(nbins=10, thres=2., qvalue=0.03, phiwidth=2., show=True, save=False):
    """Calculate a new mask that masks streaks.
    
    Args:
        nbins (int): the number of time bins. Instead of averaging all images, the function is 
            looping ofer nbin time bins.
        thres: (float): the threshold to identify streaks.
        phiwidth (float): the angular width of a masked streak.
        show (bool): if a plot should be created showing the results.
        
    Returns:
        newmask (np.ndarray): the new mask.
        
    """
    newmask = d.setup.mask.copy()
    
    nframes = d.meta.loc[database_id, 'nframes']
    stepsize = nframes // nbins
    
    phimap = d.setup.ai.chiArray()*180/np.pi
    
    for first in range(0, nframes-stepsize, stepsize):
        avr = d.get_series(database_id, first=first, last=first+stepsize, verbose=False)
        avr = avr.mean(0)
        I, q, p = d.setup.ai.integrate2d(avr, 200, mask=~newmask, method="cython", )
        qbin = np.argmin(np.abs(q-qvalue))

        phiinds = I[:,qbin] > thres
        for phi in p[phiinds]:
            ind = (phimap < (phi+phiwidth/2)) & (phimap > (phi-phiwidth/2))
            newmask[ind] = 0
            
        if show:
            I, q, p = d.setup.ai.integrate2d(avr, 200, mask=~newmask, method="cython", dummy=np.nan)
            if first == 0:  
                fig, axs = plt.subplots(3, 1, figsize=(8,8), constrained_layout=True)
                
                a = axs.flat[1]
                my_cmap = plt.get_cmap('inferno')
                pc = a.imshow(I.T, norm=LogNorm(), extent=[p[-1], p[0], q[0], q[-1]], origin='lower', aspect="auto")
                a.set_xlabel('phi')
                a.set_ylabel('q (nm-1)')
                
                a = axs.flat[0]
                avr[newmask==0] = np.nan
                im = a.imshow(avr, norm=LogNorm())
                
                a = axs.flat[2]
                ln, = a.plot(I[:,qbin], 'o', ms=2)
                a.hlines(thres, 0, 360, ls='--', color='k')
                a.set_xlabel('phi')
                a.set_ylabel(f'intensity at q={q[qbin]:.2}nm-1')
                fig.canvas.draw()
                time.sleep(5)
            else:
                pc.set_data(I.T)
                
                avr[newmask==0] = np.nan
                im.set_data(avr) 
                
                ln.set_ydata(I[:,qbin])
                axs.flat[2].autoscale()

            fig.canvas.draw()
            
    if not os.path.isdir('../03-masks/'):
        os.mkdir('03-masks')
    if save:
        filename = f'../03-masks/{sample}_{run_series:05d}_{database_id:02d}.npy'
        np.save(filename, newmask)
        print(f"New mask saved as: {filename}")

    return newmask

# Load data and initialize Xana

In [2]:
sample = 'HydLys_0p25_2' # runs 168 
run_series = 8
maskfile = '../03-masks/mask-eiger4m-05-grande.npy'
setup = '../04-setups/eiger4m-03-powders-phis-03.pkl'

In [ ]:
d = Xana(fmtstr='p10_eiger_h5', # format string that defines how the date are read
           sample=sample,  # optional: sample name used in data base
           detector='eiger4m', # file that conatains the mask 
           maskfile=maskfile, # optional, but necessary for analysis
           setupfile=setup)

In [ ]:
d.connect(f'{rawdir}{sample}_{run_series:05d}')
d.meta

## Read 2d images and plot

In [ ]:
idx = 0
database_id = d.meta[d.meta['series']==run_series].index.values[idx]
filename = d.meta.loc[database_id, 'master']

In [ ]:
avr, V = d.get_series(database_id, method='average', verbose=True, last=1000)
print(f"Loaded data with shape {np.shape(avr)}")

In [ ]:
figure(figsize=(7,3), constrained_layout=True)
imshow(d.setup.mask)

In [ ]:
avr[d.setup.mask==0] = np.nan

figure(figsize=(7,3), constrained_layout=True)
imshow(avr, norm=LogNorm())

## Mask streaks

`d.setup.ai.integrate2d` can be used to calculate the azimhuthal integration. Remember that this step requires that you called `d.setup.make` before (we did this a couple of cells before). Look at the intensity fluctuations in the second plot and define a `threshold`. Values above `threshold` will be masked as streaks. You can also define a `qvalue` that sets the q-section (white dashed line) that should be used for identifying a streak.

In [ ]:
qvalue = 0.09 # set q value you want to look at

I, q, p = d.setup.ai.integrate2d(avr, 200, mask=~d.setup.mask, method="cython", )
qbin = np.argmin(np.abs(q-qvalue))

fig, (ax1, ax2) = subplots(2, 1, figsize=(7,5), constrained_layout=True, sharex=True)
pc = ax1.pcolor(p, q, I.T, norm=LogNorm())#, shading='auto')
ax1.set_ylabel('q (nm-1)')
ax1.hlines(q[qbin], -175, 175, ls='--', color='w')

threshold = np.quantile(I[:,qbin], .99) # here set threshold

ax2.plot(p, I[:,qbin], label=f"q = {q[qbin]:.3f} nm-1")
ax2.set_ylabel('intensity')
ax2.set_xlabel('phi')

ax2.hlines(threshold, -175, 175, ls='--', color='k')
ax2.legend()

In [ ]:
newmask = mask_streaks(thres=threshold, qvalue=qvalue, save=True)

In [ ]:
d.setup.mask = newmask # we update the mask

In [ ]:
d.mksavdir('../04-setups/') # folder to save the setup with the new mask
d.savesetup(f'setup-{sample}_{run_series:05d}_dbid{database_id:02d}-phis')

# Analyse

**Load mask and setup**

In [ ]:
maskfile = f"../03-masks/{sample}_{run_series:05d}_{database_id:02d}.npy"
setupfile = f"../04-setups/setup-{sample}_{run_series:05d}_dbid{database_id:02d}-phis.pkl" # setup

d = Xana(fmtstr='p10_eiger_h5', 
           sample=sample,
           detector='eiger4m', 
           maskfile=maskfile,
           setupfile=setupfile)

In [ ]:
d.connect(f'{rawdir}{sample}_{run_series:05d}')
d.mksavdir('../05-analysis-phis/')

database_id = d.meta[d.meta['series']==run_series].index.values[idx]
display(d.meta.loc[[database_id]])

**Run analysis**

In [ ]:
d.analyze(database_id, 'xpcs', verbose=True, nprocs=4, nread_procs=1, norm='symmetric',
          saxs='compute', first=10, #last=5000,
          twotime_par=list(np.arange(len(d.setup.qroi)))

In [ ]:
d.analyze(database_id, 'saxs', verbose=True, first=10)